In [1]:
import pandas as pd
import json
import re
import os
import subprocess
import bibtexparser
from tqdm import tqdm
import re

In [ ]:
#TODO: write a function that normalizes the names of the authors from our annotations
# to fit the format generated by anystyle.io check below for examples and look for the pattern

In [2]:
def file_finder(file_str: str) -> str:
    """
    This function takes a file name and returns the path to the file in the all_data_articles.
    """
    title_doi = "../data/titles_doi.csv"
    folder_path = "../all_data_articles"
    
    #extract the doi from the file name
    doi = file_str.split("_")[-1].split(".")[0]

    # find the row in the csv file where the doi column ends with the doi
    df = pd.read_csv(title_doi)
    doi_row = df[df["DOI"].str.endswith(doi)]

    # extract the title from the row
    title_json = doi_row["Title"].values[0].replace(" ", "_") + ".json"

    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and filename.startswith(title_json[:int(len(title_json)/3)]):
            return filename


In [3]:
def load_annotations(file_str: str) -> pd.DataFrame:
    """
    This function takes a file name and returns the annotations from the file.
    And also replaces missing values with None.
    """
    folder_path = "../data/annotated"

    file_path = os.path.join(folder_path, file_str)
    df = pd.read_excel(file_path)

    # replace missing values with None
    df = df.where(pd.notnull(df), None)

    # replace values marked with nan with None
    df = df.replace("nan", None)
    
    return df

In [4]:
def format_author_name(name):
    # TODO: probably need to handle more cases
    if name is None:
        return None
    if ' and ' in name:
        # Handle multiple authors
        authors = name.split(' and ')
        formatted_authors = [format_author_name(author) for author in authors]
        return ' and '.join(formatted_authors)
    else:
        parts = name.split()
        # Handle case where there is a middle initial
        if len(parts) == 3:
            return f"{parts[1]}, {parts[0]} {parts[2]}"
        # Handle case where there is no middle initial
        elif len(parts) == 2:
            return f"{parts[1]}, {parts[0]}"
        else:
            return name

In [5]:
def df_to_triplets(df: pd.DataFrame, format_author = True) -> set:
    """
    This function takes a dataframe and returns a set of triplets.
    """
    triplets = set()
    for i in range(len(df)):
        if format_author:
            triplet = (df.iloc[i]["Footnote"], format_author_name(df.iloc[i]["Authors"]), df.iloc[i]["Title"])
        else:
            triplet = (df.iloc[i]["Footnote"], df.iloc[i]["Authors"], df.iloc[i]["Title"])
        triplets.add(triplet)
    return triplets

In [6]:
def dict_to_triplets(extraction: dict) -> set:
    """
    Converts a dictionary of footnotes to a set of triplets
    """
    triplets = set()

    for number, references in extraction.items():
        for reference in references:
            author = reference[0]
            title = reference[1]

            if author == "":
                author = None

            if title == "":
                title = None
            
            triplets.add((int(number), author, title))

    return triplets

In [7]:
def information_extraction(file_path: str) -> set:
    """
    This function takes a file path and returns a set of triplets.
    """
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    extraction = {}

    prev_footnote = None

    for number, footnote in tqdm(article["footnotes"].items()):

        # If the footnote is ibid, use the previous footnote
        if footnote.startswith("Ibid"):
            footnote = prev_footnote

        # Store the footnote for the next iteration
        prev_footnote = footnote
        
        references = re.split(';',footnote)
        
        author_title_list = []

        for reference in references:

            command = ['ruby', 'anystyle.rb', str(reference)]
            bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
            parsed_bibtex = bibtexparser.loads(bibtex).entries
            if parsed_bibtex:
                parsed_bibtex = parsed_bibtex[0]
            else:
                print(f"No valid BibTeX entry found in: {bibtex}, set to empty dict")
                parsed_bibtex = {}

            #print(parsed_bibtex)

            # Extract title, prioritizing 'booktitle' if both 'title' and 'booktitle' are present
            title = parsed_bibtex.get('booktitle', parsed_bibtex.get('title', None))

            # Extract author information
            author = parsed_bibtex.get('author', None)

            # Append author and title pair to the list
            author_title_list.append([author, title])

        # Store the list in the extraction dictionary with the footnote number as the key
        extraction[number] = author_title_list

    return dict_to_triplets(extraction)

In [11]:
def calculate_scores(triplets, extractions):
    TP = len(triplets & extractions)  # Intersection of triplets and extractions
    FP = len(extractions - triplets)  # Elements in extractions but not in triplets
    FN = len(triplets - extractions)  # Elements in triplets but not in extractions

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return recall, precision, f_score

In [195]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=True)
    extraction = information_extraction(title_json)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [01:05<00:00,  2.25it/s]


Recall: 0.2268041237113402
Precision: 0.2430939226519337
F-Score: 0.23466666666666663
-------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


 98%|█████████▊| 121/124 [01:02<00:01,  1.84it/s]

No valid BibTeX entry found in: @thesis{hope2023a,
  author = {Hope, M.},
  title = {Underwriting Risk: Trade, War, Insurance, and Legal Institutions in Eighteenth-Century France and Its Empire},
  date = {2023},
  pages = {8},
  type = {(Yale Univ. Ph.D. thesis,}
}
, set to empty dict


...

Recall: 0.2537313432835821
Precision: 0.281767955801105
F-Score: 0.2670157068062827
-------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


 22%|██▏       | 22/100 [00:09<00:31,  2.49it/s]

No valid BibTeX entry found in: @patent{pollard1974a,
  author = {Pollard and C. Ross, Edward IV (London},
  title = {Lord Fitzhugh’s Rising’},
  pages = {171},
  date = {1974},
  note = {also counts eleven women. Both cite the same pages of the Patent Rolls given above.}
}
, set to empty dict


100%|██████████| 100/100 [00:46<00:00,  2.14it/s]

Recall: 0.05106382978723404
Precision: 0.0967741935483871
F-Score: 0.06685236768802229
-------------------------




In [47]:
# TODO: implement a simple approach with a tagger or just regrex to extract the author and title, and a simple split with ";"

def extract_citations(file_path: str) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = re.split(";", footnote_text)
        
        for citation_text in individual_citations:
            # Regular expression to extract authors and titles
            # TODO: try a better pattern
            pattern = re.compile(r'^(.+?),\s+(.+?)[,|(]')

            match = pattern.match(citation_text)
            
            if match:
                author = match.group(1)
                title = match.group(2)
                citations.add((int(footnote_number), author, title))


    return citations

 

In [49]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = extract_citations(title_json)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:00<00:00, 461529.36it/s]




Recall: 0.005154639175257732
Precision: 0.005780346820809248
F-Score: 0.005449591280653951
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:00<00:00, 347423.98it/s]




Recall: 0.15306122448979592
Precision: 0.189873417721519
F-Score: 0.1694915254237288
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.1382488479262673
Precision: 0.16042780748663102
F-Score: 0.1485148514851485
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx



100%|██████████| 115/115 [00:00<00:00, 338250.32it/s]




Recall: 0.009950248756218905
Precision: 0.011764705882352941
F-Score: 0.010781671159029648
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:00<00:00, 380608.35it/s]



Recall: 0.05106382978723404
Precision: 0.10526315789473684
F-Score: 0.06876790830945557
--------------------------------------------------




In [31]:
def extract_triplets(triplets):
    return list(filter(lambda triplet: 30 < triplet[0] < 40, triplets))

In [148]:
extract_triplets(extraction)

[(32, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (38, 'Madge and Harrisson, Mass Observation', None),
 (37, None, 'First Year’s Work, 1937–1938'),
 (31, 'Warner, R.', 'Mind in Chains'),
 (33, 'Madge and Harrisson, Mass Observation', None),
 (36, None, 'First Year’s Work, 1937–1938'),
 (39, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (35,
  'Jahoda-Lazarsfeld, M. and Zeisel, H.',
  'Die Arbeitslosen von Marienthal'),
 (34,
  'Lynd, R.S. and Lynd, H.M. and Middletown and Igo, S.',
  'For the best overview of the study'),
 (35, 'Madge and Harrisson, Mass Observation', None)]

In [149]:
extract_triplets(triplets)

[(32, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (37, 'C. Madge and T. Harrisson', 'First Year’s Work, 1937–1938'),
 (34,
  'S. Igo',
  'The Averaged American: Surveys, Citizens, and the Making of a Mass Public'),
 (34, 'R.S. Lynd and H.M. Lynd', 'Middletown'),
 (36, 'C. Madge and T. Harrisson', 'First Year’s Work, 1937–1938'),
 (31, 'R. Warner', 'Education'),
 (33, 'Madge and Harrisson', 'Mass Observation'),
 (39, 'Madge', 'Press, Radio, and Social Consciousness'),
 (35, 'M. Jahoda-Lazarsfeld and H. Zeisel', 'Die Arbeitslosen von Marienthal'),
 (38, 'Madge and Harrisson', 'Mass Observation')]

In [ ]:
#(113, 'M. Young and P. Willmott', 'Family and Kinship in East London'),
#('113', 'Young, M. and Willmott, P.', 'Family and Kinship in East London'),